<a href="https://colab.research.google.com/github/han7653/Somke/blob/main/%EC%97%B0%EA%B8%B0%ED%83%91%EC%A7%80%EB%AA%A8%EB%8D%B8_20240502_robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#연기탐지모델

2024년 05 월 02일 로봇응용 SW전문가 과정 -한승헌



##연기탐지모델이란?


1. 화재로 인한 연기가 일어났을때 확인가능
2. 그러므로 빠른 대처 가능

##연기 탐지를 위한 데이터 세트

https://public.roboflow.com/object-detection/wildfire-smoke/

##사용 AI모델

1.yoloV5

.가장 빠른 인식률로 실시간 탐지에 유리하다.

. CNN알고리즘 중 가장 활발하게 연구활동이 이루어지고 있다.

2.yoloV5는 객체 탐지 방법론의 진보를 나타낸다.

##AI 응용 SW를 개발하는 5단계
     1.What? 내가 어떤 모델을 개발할것인지
     2.DataSet을 준비한다. -양질의 DataSet은 SW품질을 좌우한다.
     3.적합한 Pre-Trained된 AI 모델을 선택한다.

-오늘 공부할 내용

     4.FIne-Tunning : 2에서 준비한 DataSet을 이용해서 3을 Dustomizing한다.

-마지막으로
    
     5.응용 SW를 제작한다.
      .WebApp - 이전에 실습했던 TM을 기억한다.
      .nativeApp -python과 pyQT로 작성했던 기억


#YOLO Clone from github

##작업 폴더를 준비하고 이동한다.

In [ ]:
%cd /content

/content


##github에서 복사해오기

In [ ]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


##개발환경을 install한다

In [ ]:
%pwd

'/content'

In [ ]:
%cd yolov5
%pip install -r requirements.txt

/content/yolov5
  Using cached GitPython-3.1.43-py3-none-any.whl (207 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached ultralytics-8.2.6-py3-none-any.whl (755 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)


##데이터세트 가져오기

위치는 https://public.roboflow.com/object-detection/wildfire-smoke/

In [ ]:
#폴더를 만든다
%mkdir /content/yolov5/smoke
%cd /content/yolov5/smoke

mkdir: cannot create directory ‘/content/yolov5/smoke’: File exists
/content/yolov5/smoke


In [ ]:
!curl -L curl -L curl -L "https://public.roboflow.com/ds/G24GbV9hy1?key=RVNXbCqui2" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: curl
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: curl
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   910  100   910    0     0   2936      0 --:--:-- --:--:-- --:--:--  2944
100 27.6M  100 27.6M    0     0  19.3M      0  0:00:01  0:00:01 --:--:-- 42.7M
Archive:  roboflow.zip
replace test/images/ck0rqdyeq4kxa08633xv2wqe1_jpeg.rf.30df6babe258cdc1d5a4e042262665e4.jpg? [y]es, [n]o, [A]ll,

원본 DataSet의 구조를 알아보기

In [ ]:
!cat /content/yolov5/smoke/data.yaml


test: ./smoke/test/images
train: ./smoke/train/images
val: ./smoke/valid/images

nc: 1
names: ['smoke']


##데이터의 구성을 확인한다

In [ ]:
from glob import glob
train_img_list = glob('/content/yolov5/smoke/train/images/*.jpg')
test_img_list = glob('/content/yolov5/smoke/valid/images/*.jpg')
valid_img_list = glob('/content/yolov5/smoke/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

516 147 147


##데이터 list를 txt파일로 만든다

In [ ]:
import yaml
with open('/content/yolov5/smoke/train.txt','w') as f:
    f.write('\n'.join(train_img_list) + '\n')
with open('/content/yolov5/smoke/test.txt','w') as f:
    f.write('\n'.join(test_img_list) + '\n')
with open('/content/yolov5/smoke/val.txt','w') as f:
    f.write('\n'.join(valid_img_list) + '\n')

##프로그램 중에 사용할 함수를 정의합니다.

writetemplate이라는 함수

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

##이전에 확인했던 data.yaml의 내용을 수정합니다.

내용은 기존의 내용중에 데이터의 위치를 수정

**참고1의 내용을 복사해서 경로만 수정함

In [ ]:
%%writetemplate /content/yolov5/smoke/data.yaml

test: ./smoke/test/images
train: ./smoke/train/images
val: ./smoke/valid/images

nc: 1
names: ['smoke']

In [ ]:
%cat /content/yolov5/smoke/data.yaml


test: ./smoke/test/images
train: ./smoke/train/images
val: ./smoke/valid/images

nc: 1
names: ['smoke']


##파인튜닝된 모델에서 사용할 custom_yolov5s.yaml을 만들기

1.nc:3이라고 적힌 것을 읽어서 변수에 저장

2.yolov5s.yaml 의 내용을 cat으로 확인하고 복사

3.writeTemplate함수를 이용해서 새로 작성 nc:80이라고 적힌 부분만 nc:{num_classes}라는 변수로 대체

In [ ]:
import yaml

with open('/content/yolov5/smoke/data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

print (f'num_classes: = {num_classes}')

%cat /content/yolov5/models/yolov5s.yaml

num_classes: = 1
# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "neare

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes} # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]

##fine_tunning 모델에서 사용할 custom_yolov5s.yaml의 내용

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml


# Parameters
nc: 1 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-

데이터 세트의 준비가 끝났습니다.

##학습

1.학습에 사용되는 train.py는 gitclone했을때 생성되었으며

2.코드 수정없이 다음 hyperParameter를 지정해서 사용한다.

하이퍼 파라메터는 다음과 같다.

1.img: 입력 이미지 크기 정의

2.batch: 배치 크기 결정

3.epochs: 학습 기간 개수 정의

4.data: yaml 파일 경로

5.cfg: 모델 구성 지정

6.weights: 가중치에 대한 경로 지정

7.name: 결과 이름

8.nosave: 최종 체크포인트만 저장

9.cache: 빠른 학습을 위한 이미지 캐시

##학습시작

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data ./smoke/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name smoke_result --cache

/content/yolov5
2024-05-02 07:39:47.199355: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 07:39:47.199427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 07:39:47.201776: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=./models/custom_yolov5s.yaml, data=./smoke/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, m

##tensorBoard를 이용해서 결과 확인

현재 폴더는 틀림없이 yolov5이다

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

##우리가 최종적으로 갖게 되는 가중치 파일은

/content/yolov5/runs/train/_result/weights 폴더에

다음 2개의 파일이 있다.

best.pt - epoch중 가장 성능이 좋은 모델

last.pt - 가장 마지막에 만들어진 모델

그래서 best.pt를 내 컴퓨터 또는 구글 드라이브에 다운 로드하여 두기로 한다.

##구글드라이브 연결하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

복사해두기

In [ ]:
!cp /content/yolov5/runs/train/_result/weights/best.pt /content/drive/MyDrive/yolov5Models/smoke.pt

##Inference 성능시험

유튜브의 동영상을 다운로드해서 그 동영상내의 Object를 detection 할것이다. 그리고 결과는 mp4 파일로 저장한다.

##chatGPT를 이용해서 유튜브 동영상을 다운로드 받게 도움 받는다

동영상은 임의의 것으로 모델의 성능 평가에 적합한것을 사용

In [ ]:
!pip install -U yt-dlp
!yt-dlp -f 'best[ext=mp4]' --output "input_video.mp4" "hhttps://www.youtube.com/watch?v=bUfwUnin9DE"

In [ ]:
%pwd

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/_result/weights/best.pt --img 640 --conf 0.25 --source input_video.mp4